In [40]:
import re
import pandas as pd
from datetime import datetime

df = pd.read_excel('caffe.xlsx')

# Adjusting quantity

In [41]:
def adjust_quantity(desc, qty, discount):
    pattern = r"(\d+)\+(\d+)"
    match = re.search(pattern, desc)
    if match:
        x, y = map(int, match.groups())
        qty = qty * x + y
        desc = re.sub(pattern, "", desc).strip()
        discount = match.group(0)
    return desc, qty, discount

In [42]:
df["Discount Type"] = ""

df[["Item Desc", "Quantity", "Discount Type"]] = df.apply(
    lambda row: pd.Series(adjust_quantity(row["Item Desc"], row["Quantity"], row["Discount Type"])),
    axis=1
)

# Add weekdays

In [43]:
def add_day_of_week(date):
    wd = ['Monday', 'Tuesday', 'Wednesday', 'Thursday', 'Friday', 'Saturday', 'Sunday']
    return wd[date.weekday()]

In [44]:
df["Week day"] = df["Date"].apply(add_day_of_week)

df.to_csv('caffe_change.csv', index=False)